In [ ]:
%%bash
pip install numpy
pip install pandas
pip install seaborn
pip install mlxtend
pip install arch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import re
from textwrap import wrap
import matplotlib.ticker
import os
import seaborn as sns
sns.color_palette("bright", 8)
sns.set_palette("bright")
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_style("whitegrid", {'axes.grid' : False})

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11.7,8.27)

%matplotlib inline

In [ ]:
def append_vars(ds, cols):
    v=ds[cols]
    X=ds.drop(columns=cols)
    return pd.concat([X.assign(cfu=ds[col], name=ds['year'].apply(lambda y: f"{col} {y}")) for col in cols]).dropna()

In [ ]:
def save_plot(fname):
    plt.savefig(f"plots/{fname}.svg")
    plt.savefig(f"plots/{fname}.png")

In [ ]:
import matplotlib.ticker as mtick

def make_dfs(df,fname):
    colors = ['#000A9C', '#5B0094', '#005903', '#68DFFC', '#4275FF', '#C973FF', '#A2FCA5']
    fig, ax = plt.subplots(figsize=(16,8))
    #sns.set_palette(colors)
    ax=sns.stripplot(x='week', y='cfu', hue='name', data=df,  alpha=1.0, size=12, linewidth=1, edgecolor='w', palette=colors)
    #ax=sns.scatterplot(x='week', y='cfu', hue='name', data=c,  alpha=0.8, size= 14, linewidth=1, edgecolor='w')
    plt.axvline(x=1.42, color='b', linestyle='-.', alpha=0.8)
    plt.axvline(x=3.7, color='b', linestyle='-.', alpha=0.8)
    ax.text(2.56, 1960,  "Festival 2016", ha="center", va="center", size=16, color='b')
    ax.annotate("", xy=(1.42, 1900), xycoords='data',
               xytext=(3.7, 1900), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='b', alpha=0.8))
    plt.axvline(x=2.4, color='r', linestyle='-.', alpha=0.8)
    plt.axvline(x=4.58, color='r', linestyle='-.', alpha=0.8)
    ax.text(3.4, 2260,  "Festival 2017", ha="center", va="center", size=16, color='r')
    ax.annotate("", xy=(2.4, 2200), xycoords='data',
               xytext=(4.58, 2200), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='r', alpha=0.8))
    ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
    ax.set_xlabel("Sample weeks 2016, 2017 and 2020", size=16, labelpad=20)
    ax.set_ylabel("Colony forming units per 100mL", size=16, labelpad=20)
    plt.suptitle("Festival and occurrence of bacteria in lake water" , fontsize=16, family='sans')
    plt.title("Colonies grouped by year and week number", fontsize=16, family='sans', y=1.03)
    ax.legend(loc='upper right', fontsize=14)
    save_plot(fname)

In [ ]:
A=pd.read_pickle("data/allyears-avg_plate.pkl")
A['Date_dt'] = pd.to_datetime(A['Date'])
R=pd.read_csv('data/rain_all.csv')
A2=append_vars(A,['Bioindicator', 'Bioindicator UV+', 'Coliform'])

In [ ]:
A

## Year over year 2016 2017 and 2020

### Bioindicator + Coliform

In [ ]:
#sns.set_palette("muted")
make_dfs(A2,'YEAR_OVER_YEAR_CFUINCREASE_JAZZ_2016_2017_2020')

In [ ]:
#make_dfs(A2[A2.name.str.contains('Bioindicator')],'YEAR_OVER_YEAR_CFUINCREASE_JAZZ_2016_2017_2020')

## Year over year 2016 2017 

In [ ]:
def make_dfs_2016_2017(df,fname):
    #colors = ['#00008b', '#1e90ff', '#40e0d0', 'g', 'y']
    colors = ['#000A9C', '#5B0094', '#68DFFC', '#4275FF', '#C973FF']
    fig, ax = plt.subplots(figsize=(16,8))
    #sns.set_palette(colors)
    ax=sns.stripplot(x='week', y='cfu', hue='name', data=df,  alpha=0.8, size=12, linewidth=1, edgecolor='w', palette=colors)
   #ax=sns.scatterplot(x='week', y='cfu', hue='name', data=c,  alpha=0.8, size= 14, linewidth=1, edgecolor='w')
    plt.axvline(x=1.42, color='b', linestyle='-.', alpha=0.8)
    plt.axvline(x=3.7, color='b', linestyle='-.', alpha=0.8)
    ax.text(2.56, 600,  "Festival 2016", ha="center", va="center", size=14, color='b')
    ax.annotate("", xy=(1.42, 580), xycoords='data',
               xytext=(3.7, 580), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='b', alpha=0.8))
    plt.axvline(x=2.4, color='r', linestyle='-.', alpha=0.8)
    plt.axvline(x=4.58, color='r', linestyle='-.', alpha=0.8)
    ax.text(3.4, 800,  "Festival 2017", ha="center", va="center", size=14, color='r')
    ax.annotate("", xy=(2.4, 780), xycoords='data',
               xytext=(4.58, 780), textcoords='data',
               arrowprops=dict(arrowstyle="<|-|>", connectionstyle="arc3", color='r', alpha=0.8))
    ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
    ax.set_xlabel("Sample weeks 2016, 2017", size=16, labelpad=20)
    ax.set_ylabel("Colony forming units per 100mL", size=16, labelpad=20)
    plt.suptitle("Festival and occurrence of bacteria in lake water" , fontsize=16, family='sans')
    plt.title("Colonies grouped by year and week number", fontsize=16, family='sans', y=1.03)
    ax.legend(loc='upper right', fontsize=14)
    save_plot(fname)

In [ ]:
make_dfs_2016_2017(A2.query('year != 20'),'YEAR_OVER_YEAR_CFUINCREASE_JAZZ_2016_2017')

## Problem with 0 values

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

A=pd.read_pickle("data/allyears.pkl")


## Rain plots

In [ ]:
def plot_with_rain(df,col_name, rain, sup_title, title, filename):
    mindate=df['Date'].min()
    R=rain.query(f"Date>'{mindate}'")
    collss=['red', 'blue', 'teal']
    markers =['x','o','s']
    
    fig, ax1 = plt.subplots(figsize=(15,8), edgecolor='midnightblue')
    
    ax1.bar(pd.to_datetime(R['Date']).values, R['Rain'].values, color='lightskyblue', width=0.99, alpha=0.4, label="mm of rain")
    
    ax2 = ax1.twinx() 

    ylim = [1]
    for i, n in enumerate(df['Location'].unique()):
        yn=df.query(f"Location=='{n}'")
        maxn = yn[col_name].max()
        ratio = yn[col_name].std()/maxn
        for index, row in yn.iterrows():
            t, c, j = ax2.errorbar(row['Date_dt'], row[col_name], yerr=row[col_name]*ratio, capsize=2, alpha=0.4, ecolor='magenta',)
            for k in j:
                k.set_linestyle('--')
            for mm, cap in enumerate(c):
                cap.set_marker('o')
        ax2.scatter(yn['Date_dt'], yn[col_name], label=n, color=collss[i], marker=markers[i], s=60, linewidth=3)
        if maxn > max(ylim):
            ylim.append(maxn)
        ax2.set_ylim(bottom=0, top=max(ylim)+100)

 
      
    for pos in ['bottom', 'top', 'left', 'right']:
        ax1.spines[pos].set_color('midnightblue')
    ax1.grid(which='major', color='midnightblue', linestyle='--', alpha=0.2, axis='y')
    ax1.tick_params(axis='both', which='both', color='midnightblue')
    
    
    ax2.set_ylabel('$\itE. coli$ colony forming units per 100mL', labelpad=10, fontsize=14)
    #ax2.yaxis.set_label_position("left")
    ax2.yaxis.tick_left()
    ax2.yaxis.set_label_position("left")

    ax1.set_ylabel('Millimeters of rain per day', labelpad=10, fontsize=14)
    #ax1.yaxis.set_label_position("right")
    ax1.yaxis.tick_right()
    ax1.yaxis.set_label_position("right")
    
    fig.legend( loc=(0.74, 0.55), frameon=True, fontsize=12, edgecolor='midnightblue', framealpha=0.8 )
    
    plt.xticks(df['Date'].unique(), [f"Week {i+1}" for i in range(8)], fontsize=11 )
    plt.suptitle(sup_title, fontsize=16, family='sans')
    plt.title(title, fontsize=16, family='sans', y=1.03)
    plt.subplots_adjust(top=0.85)
    save_plot(filename)
    plt.show()

In [ ]:
A.query('year == 20')

In [ ]:
plot_with_rain(A.query('year == 20'),'total',R.query('year == 20'),
               'Weekly average total colony forming units',
               'June - Aug 2020; medium: Easygel+, 24hrs, n={}'.format(len(A.query('year == 20'))),
               '2020_AVG_Total_CFU_week')

In [ ]:
plot_with_rain(A.query('year == 20'),'Bioindicator',R.query('year == 20'),
                'Weekly average Bioindicator colony forming units',
               'June - Aug 2020; medium: Easygel+, 24hrs, n={}'.format(len(A.query('year == 20'))),
                '2020_AVG_Bioindicator_CFU_week')

In [ ]:
plot_with_rain(A.query('year == 17'),'total',R.query('year == 17'),  
               'Weekly average total colony forming units', 
               'June - July 2017; medium: Easygel+, 24hrs, n={}'.format(len(A.query('year == 17'))), 
               '2017_AVG_Total_CFU_week')

In [ ]:
plot_with_rain(A.query('year == 17'),'Bioindicator',R.query('year == 17'), 
               'Weekly average Bioindicator colony forming units',
               'June - July 2017; medium: Easygel+, 24hrs, n={}'.format(len(A.query('year == 17'))),
               '2017_AVG_BigBlue_CFU_week')

In [ ]:
plot_with_rain(A.query('year == 17'),'Bioindicator UV+',R.query('year == 17'),
               'Weekly average UV+ Bioindicator colony forming units',
               'June - july 2017; medium: Easygel+, 24hrs, n={}'.format(len(A.query('year == 17'))),
               '2017_AVG_UltraViolet_CFU_week')

In [ ]:
plot_with_rain(A.query('year == 16'),'total',R.query('year == 16'),
               'Weekly average total colony forming units', 
                'June - July 2016; medium: Easygel, 24hrs, n={}'.format(len(A.query('year == 16'))), 
                '2016_AVG_Total_CFU_week_16')

In [ ]:
plot_with_rain(A.query('year == 16'),'Bioindicator',R.query('year == 16'),
               'Weekly average Bioindicator colony forming units',
                'June - August 2016; medium: Easygel, 24hrs, n={}'.format(len(A.query('year == 16'))),
                '2016_AVG_BigBlue_CFU_week')

## Rain 72hours sum graph

In [ ]:
rain_sum_df = A2.groupby(['name','Date']).sum().reset_index()
rain_sum_df

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.ticker as mticker


def make_rain_sum(df, fname, colors):
    #colors = ['#00008b', '#8b008b', '#468b00', '#add8e6'] #, '#ffb6c1']
    #colors = ['#000A9C', '#5B0094', '#005903', '#68DFFC', '#4275FF', '#C973FF', '#A2FCA5']
    fig, ax = plt.subplots(figsize=(15,8))
    
    ax=sns.scatterplot(x='rain', y='cfu', hue='name', data=df,  alpha=1, s=100, linewidth=1, edgecolor='w', palette=colors, legend='full')

    ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
    ax.set_xlabel("Montreux 72 hour rainfall total in millimiters. 2016, 2017 and 2020", size=16, labelpad=20)
    ax.set_ylabel("Colony forming units per 100mL", size=16, labelpad=20)
    plt.suptitle("Rainfall and the occurrence of coliform bacteria in laker water" , fontsize=16, family='sans')
    plt.title("Grouped by 72 hours rainfall total and colony color", fontsize=16, family='sans', y=1.03)
    ax.legend(loc='upper right', fontsize=14)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(4))#mticker.MultipleLocator(1))

    save_plot(fname)

In [ ]:
make_rain_sum(rain_sum_df[rain_sum_df.name.str.contains('Bioindicator')].round(decimals=5), 
              'Rainfall_sum_2016_2017_2020', 
              ['#000A9C', '#5B0094', '#005903', '#68DFFC'])


In [ ]:
def make_rain_sum_2016_2017(df, fname, colors):
    #colors = ['#000A9C', '#5B0094', '#68DFFC', '#4275FF', '#C973FF']
    #colors = ['#00008b', '#8b008b', '#008000', '#468b00', '#add8e6', '#ffb6c1', '#90ee90']
    fig, ax = plt.subplots(figsize=(15,8))
    
    ax=sns.scatterplot(x='rain', y='cfu', hue='name', data=df,  alpha=1, s=100, linewidth=1, edgecolor='w', palette=colors, legend='full')

    ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
    ax.set_xlabel("Montreux 72 hour rainfall total in millimiters. 2016 and 2017", size=16, labelpad=20)
    ax.set_ylabel("Colony forming units per 100mL", size=16, labelpad=20)
    plt.suptitle("Rainfall and the occurrence of coliform bacteria in laker water" , fontsize=16, family='sans')
    plt.title("Grouped by 72 hours rainfall total and colony color", fontsize=16, family='sans', y=1.03)
    ax.legend(loc='upper right', fontsize=14)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(4))#mticker.MultipleLocator(1))

    save_plot(fname)

In [ ]:
make_rain_sum_2016_2017(rain_sum_df[rain_sum_df.name.str.contains('Bioindicator')].query('year != 60').round(decimals=5), 
                        'Rainfall_sum_2016_2017', 
                        ['#000A9C', '#5B0094', '#005903'])

In [ ]:
make_rain_sum_2016_2017(rain_sum_df.query('year != 60').round(decimals=2), 
                        'Rainfall_sum_2016_2017', 
                        ['#000A9C', '#5B0094', '#005903', '#68DFFC', '#4275FF'])

## Results before, during and after

In [ ]:
#A=pd.read_pickle("data/allyears.pkl")
A=pd.read_pickle("data/allyears-avg_plate.pkl")
A['Date_dt'] = pd.to_datetime(A['Date'])
A2=append_vars(A,['Bioindicator', 'Bioindicator UV+', 'Coliform'])

In [ ]:
festival = A2.query('year != 20')
nofestival = A2.query('year == 20').copy()
nofestival['festival'] = 'No festival'

barplot = pd.concat([festival, nofestival])
barplot = barplot[barplot.name.str.contains('Bioindicator')].query('name != "Bioindicator UV+ 17"')
barplot

## With Statistics annotations

### Permutation Test

In [ ]:
from statannot import add_stat_annotation

data = barplot.query('year != 20')
x = "festival"
y = "cfu"
order= ['before', 'during', 'after']

box_pairs=[("before", "during"), ("before", "after"), ("during", "after")]

#from scipy.stats import brunnermunzel

test_short_name = 'Permutation Test'
pvalues = [0.0001,0.1020,0.0359]

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_style("whitegrid", {'axes.grid' : False})
plt.rcParams["figure.figsize"] = (11.7,8.27)

ax = sns.boxplot(x=x, y=y, data=data, order=order, color='lightblue')
sns.stripplot(ax=ax, x=x, y=y, data=data, alpha=1, linewidth=2, order=order, color='darkblue')

ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
ax.set_xlabel("Results grouped in relation to Montreux Festival", size=16, labelpad=20)
ax.set_ylabel("$\itE. coli$ colony forming units per 100mL", size=16, labelpad=20)
plt.suptitle("Distribution of survey results summer 2016 & 2017" , fontsize=16, family='sans')
plt.title("colony forming units per 100mL; n={}".format(len(data)), fontsize=16, family='sans', y=1.03)

test_results = add_stat_annotation(ax, data=data, x=x, y=y,
                                   box_pairs=box_pairs,
                                   perform_stat_test=False, pvalues=pvalues, test_short_name=test_short_name,
                                   pvalue_thresholds=[[1.1e-4, "****"], [1.1e-3, "***"], [1.1e-2, "**"], [0.05, "*"], [1, "ns"]],text_format='star', verbose=2)
plt.savefig(f"plots/permutation_test_2016_2017_distribution.svg")
plt.savefig(f"plots/permutation_test_2016_2017_distribution.png")


test_results



In [ ]:
import matplotlib.ticker as ticker

data = barplot
x = "festival"
y = "cfu"
order= ['before', 'during', 'after', 'No festival']

box_pairs=[("before", "during"), ("before", "after"), ("during", "after"),
           ("before","No festival"), ("during","No festival"), ("after","No festival")]

test_short_name = 'Permutation Test'
pvalues = [0.0001,0.1020,0.0359, 0.0001, 0.0001,0.0001 ]

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_style("whitegrid", {'axes.grid' : False})
plt.rcParams["figure.figsize"] = (11.7,8.27)

ax = sns.boxplot(x=x, y=y, data=data, order=order, color='lightblue', showfliers=False)
sns.stripplot(ax=ax, x=x, y=y, data=data, alpha=1, linewidth=2, order=order, color='darkblue')

ax.xaxis.grid(which="major", color='b', linewidth=0.7, alpha=0.1)
ax.set_xlabel("Results grouped in relation to Montreux Festival", size=16, labelpad=20)
ax.set_ylabel("$\itE. coli$ colony forming units per 100mL", size=16, labelpad=20)
plt.suptitle("Distribution of survey results summer 2016, 2017 and 2020" , fontsize=16, family='sans')
plt.title(f"colony forming units per 100mL; n={len(barplot)}", fontsize=16, family='sans', y=1.03)
ax.yaxis.set_major_locator(ticker.MultipleLocator(100))


test_results = add_stat_annotation(ax, data=data, x=x, y=y, order=order, fontsize='medium',
                                   box_pairs=box_pairs,
                                   perform_stat_test=False, pvalues=pvalues, test_short_name=test_short_name,
                                   show_test_name=False,text_format='star',
                                   pvalue_thresholds=[[1.1e-4, "****"], [1.1e-3, "***"], [1.1e-2, "**"], [0.05, "*"], [1, "ns"]],verbose=2)
test_results

plt.savefig(f"plots/permutation_test_distribution.svg")
plt.savefig(f"plots/permutation_test_distribution.png")

The confidence interval should be extracted manually

In [ ]:
fig1, ax1 = plt.subplots()
a = ax1.boxplot(barplot.query('festival == "before"')["cfu"])
print('Median c.i. for before goes from {} to {} and a median of {}'.format(ax1.lines[0].get_xydata()[1][1]
,ax1.lines[0].get_xydata()[2][1], ax1.lines[5].get_xydata()[0][1]))

fig1, ax1 = plt.subplots()
a = ax1.boxplot(barplot.query('festival == "during"')["cfu"])
print('Median c.i. for during goes from {} to {} and a median of {}'.format(ax1.lines[0].get_xydata()[1][1]
,ax1.lines[0].get_xydata()[2][1], ax1.lines[5].get_xydata()[0][1]))

fig1, ax1 = plt.subplots()
a = ax1.boxplot(barplot.query('festival == "after"')["cfu"])
print('Median c.i. for after goes from {} to {} and a median of {}'.format(ax1.lines[0].get_xydata()[1][1]
,ax1.lines[0].get_xydata()[2][1], ax1.lines[5].get_xydata()[0][1]))

fig1, ax1 = plt.subplots()
a = ax1.boxplot(barplot.query('festival == "No festival"')["cfu"])
print('Median c.i. for No festival goes from {} to {} and a median of {}'.format(ax1.lines[0].get_xydata()[1][1]
,ax1.lines[0].get_xydata()[2][1], ax1.lines[5].get_xydata()[0][1]))
